In [671]:
import numpy as np
import pandas as pd
import io
import re
from konlpy.tag import Hannanum, Kkma, Komoran, Mecab, Okt
from gensim.models import Word2Vec

In [672]:
data = pd.read_csv('/Users/minhyuk/study/example/thesool_all.csv')
data.head()

,name,address,time,phone,menu
0,막걸리전당,서울 관악구 봉천로 461-1 1층,매일 16:00 ~ 01:00,0507-1407-7103,"전당모둠전, 옛날모둠전, 국산전두부와볶음김치, 한통다골뱅이무침, 한돈육전, 홍어를무..."
1,한식술집 돌,서울 성북구 보문로34길 87 1층,매일 17:00-24:00,0507-1336-5824,"노돈보쌈, 뭉티기, 전복내장 파스타, 메로조림, 낙지젓갈 볶음밥, 곤드레 아란치니...."
2,박가네 빈대떡,서울 종로구 종로32길 7,매일: 08:00 - 22:00,02-2264-0847,"박가네 삼합, 맷돌 빈대떡,고기 빈대떡, 해물 빈대떡, 마약김밥(2인), 육회탕탕이..."
3,종부네,서울 동작구 동작대로27가길 37 2층,화~목: 17:00-01:00 금17:00-01:00 토: 14:00-01:00 일...,010-4305-1805,"육회, 통골뱅이파무침, 오뱅새세트, 마약옥수수전, 모둠전, 치즈감자채전"
4,화실,서울 노원구 한글비석로20길 62 송림빌딩 101호,"수,목 17:00-01:00 금,토 17:00-02:00 (일,월 정기휴무)",0507-1397-2490,"불고기 트러플 크림 파케리, 청어알 들기름 냉파스타, 명란감자구이, 청어알 두부쌈,..."


In [673]:
shop_list = data[['name', 'menu']]
shop_list.head()

,name,menu
0,막걸리전당,"전당모둠전, 옛날모둠전, 국산전두부와볶음김치, 한통다골뱅이무침, 한돈육전, 홍어를무..."
1,한식술집 돌,"노돈보쌈, 뭉티기, 전복내장 파스타, 메로조림, 낙지젓갈 볶음밥, 곤드레 아란치니...."
2,박가네 빈대떡,"박가네 삼합, 맷돌 빈대떡,고기 빈대떡, 해물 빈대떡, 마약김밥(2인), 육회탕탕이..."
3,종부네,"육회, 통골뱅이파무침, 오뱅새세트, 마약옥수수전, 모둠전, 치즈감자채전"
4,화실,"불고기 트러플 크림 파케리, 청어알 들기름 냉파스타, 명란감자구이, 청어알 두부쌈,..."


In [674]:
# df = pd.DataFrame(shop_list, columns = ['name', 'menu'])
# df.head()

In [675]:
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()


def tokenizer(row):
    return kkma.nouns(row)

In [676]:
# from tqdm import tqdm


# noun_list = []
# for content in tqdm(df['menu']):
#     nouns = kkma.nouns(content)
#     noun_list.append(nouns)

# df['nouns'] = noun_list
# df.head()
shop_list['rev_token'] = shop_list['menu'].apply(tokenizer)
shop_list.head()

/var/folders/k6/8bdr75150lx2_c2xq40f94nm0000gn/T/ipykernel_46046/1638933935.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shop_list['rev_token'] = shop_list['menu'].apply(tokenizer)


,name,menu,rev_token
0,막걸리전당,"전당모둠전, 옛날모둠전, 국산전두부와볶음김치, 한통다골뱅이무침, 한돈육전, 홍어를무...","[전당, 전당모둠전, 모둠, 전, 옛날, 옛날모둠전, 국산, 전두, 전두부와, 부와..."
1,한식술집 돌,"노돈보쌈, 뭉티기, 전복내장 파스타, 메로조림, 낙지젓갈 볶음밥, 곤드레 아란치니....","[노, 노돈보쌈, 돈, 보쌈, 뭉, 뭉티, 티, 전복, 전복내장, 내장, 파스타, ..."
2,박가네 빈대떡,"박가네 삼합, 맷돌 빈대떡,고기 빈대떡, 해물 빈대떡, 마약김밥(2인), 육회탕탕이...","[박, 삼, 삼합, 합, 맷돌, 빈대떡, 고기, 해물, 마약, 마약김밥, 김밥, 2..."
3,종부네,"육회, 통골뱅이파무침, 오뱅새세트, 마약옥수수전, 모둠전, 치즈감자채전","[육회, 통, 통골뱅이파무침, 골뱅이, 파, 무침, 오, 오뱅새세트, 뱅, 새, 세..."
4,화실,"불고기 트러플 크림 파케리, 청어알 들기름 냉파스타, 명란감자구이, 청어알 두부쌈,...","[불고기, 플, 크림, 파, 리, 청어, 들기름, 냉, 냉파스타, 파스타, 명란, ..."


In [677]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# text = [" ".join(noun) for noun in df['nouns']]

# tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1, 5))
# tfidf_vectorizer.fit(text)
# vector = tfidf_vectorizer.transform(text).toarray()

In [678]:
# from sklearn.cluster import DBSCAN
# import numpy as np

# vector = np.array(vector)
# model = DBSCAN(eps=0.3, min_samples=6, metric = "cosine")
# result = model.fit_predict(vector)

# df['result'] = result
# df.head()

In [679]:
# for cluster_num in set(result):
#     if(cluster_num == -1 or cluster_num == 0):
#         continue
#     else:
#         print("cluster num : {}".format(cluster_num))
#         temp_df = df[df['result'] == cluster_num]
#         for menu in temp_df['menu']:
#             print(menu)
#         print()

In [680]:
print('최대 토큰 개수 : ', (max(len(i) for i in shop_list['rev_token'])))
print('평균 토큰 개수 : ', sum(map(len, shop_list['rev_token'])) / len(shop_list['rev_token']))

최대 토큰 개수 :  41
평균 토큰 개수 :  11.0


In [681]:
model = Word2Vec(sentences = shop_list['rev_token'], vector_size = 300, window = 5, min_count = 0, workers = 4, sg = 1)
model.wv.vectors.shape

(517, 100)

In [682]:
model_result = model.wv.most_similar('해물')
print(model_result)

[('콩나물', 0.2541230618953705), ('메', 0.25234392285346985), ('파스타', 0.22621923685073853), ('재료', 0.2259368747472763), ('동래', 0.2196091264486313), ('수육전골', 0.2183971256017685), ('개구리', 0.2135980874300003), ('치즈감자채전', 0.2095862478017807), ('갑오징어숙회', 0.20877468585968018), ('씨', 0.20823444426059723)]
